In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
# pd.set_option('display.max_columns', 20)
# pd.set_option('display.width', 1000)

movies = pd.read_csv('./data/tmdb_5000_movies.csv')
movies = movies.loc[:, ['title', 'genres', 'keywords']]

# 문자열 -> 딕셔너리 형태로 변환
movies['genres'] = movies['genres'].apply(literal_eval)
movies['keywords'] = movies['keywords'].apply(literal_eval)

# i = 0
# for row in movies['genres']:
#     genres = []
#     for ele in row:
#         genres.append(ele['name'])
#     movies['genres'][i] = genres
#     i = i + 1

# 딕셔너리 형태를 깔끔하게 장르에 해당하는 부분만 뽑아서 문자열화
# [{}, {}, {}, {}] -> [장르, 장르, 장르, 장르]
movies['genres'] = movies['genres'].apply(lambda x : [y['name'] for y in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [y['name'] for y in x])

# [장르, 장르, 장르, 장르] -> 장르 장르 장르 장르
movies['genres'] = movies['genres'].apply(lambda x : ' '.join(x))
movies['keywords'] = movies['keywords'].apply(lambda x : ' '.join(x))

print(movies)

                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                        genres  \
0     Action Adventure Fantasy Science Fiction   
1                     Adventure Fantasy Action   
2                       Action Adventure Crime   
3                  Action Crime Drama Thriller   
4             Action Adventure Science Fiction   
...                                        ...   
4798                     Action Crime Thriller   

# TFIDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range=(1, 2) 는 단어를 1개 혹은 2개 연속으로 보겠다
tfidf_vec = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(movies['keywords'])
print(tfidf_vec.vocabulary_.items())
# 4803은 영화의 개수, 276은 단어의 개수 -> 하나의 영화를 276개 열을 가진 벡터로 표현
print(tfidf_matrix.shape)

# action adventure fantasy
# adventure fantasy
# adventure
# fantasy

dict_items([('culture', 9037), ('clash', 7010), ('future', 15032), ('space', 34485), ('war', 40021), ('colony', 7433), ('society', 34214), ('travel', 38236), ('futuristic', 15087), ('romance', 31412), ('alien', 1077), ('tribe', 38408), ('planet', 27732), ('cgi', 6161), ('marine', 22347), ('soldier', 34275), ('battle', 3232), ('love', 21496), ('affair', 548), ('anti', 1760), ('power', 28328), ('relations', 30072), ('mind', 23178), ('and', 1426), ('soul', 34431), ('3d', 94), ('culture clash', 9039), ('clash future', 7012), ('future space', 15075), ('space war', 34525), ('war space', 40145), ('space colony', 34494), ('colony society', 7442), ('society space', 34240), ('space travel', 34523), ('travel futuristic', 38253), ('futuristic romance', 15094), ('romance space', 31433), ('space alien', 34487), ('alien tribe', 1148), ('tribe alien', 38409), ('alien planet', 1122), ('planet cgi', 27737), ('cgi marine', 6165), ('marine soldier', 22373), ('soldier battle', 34281), ('battle love', 3254)

In [4]:
# 유사도 행렬 (4803, 4803)
# 1, 1 (1번째 영화와 1번재 영화의 유사도)
# 1, 1 / 1, 2 / .... / 1, 4803 -> 1번째 영화와 1~4803번재 영화의 유사도
# 2, 1 / 2, 2 / .... / 2, 4803 -> 2번째 영화와 1~4803번째 영화의 유사도
# ....
# 4803, 1 / 4803, 2 / .... / 4803, 4803 -> 4803번째 영화와 1~4803번째 영화의 유사도
from sklearn.metrics.pairwise import cosine_similarity

# 4803개의 영화랑 4803개의 영화끼리 유사도를 구하겠다!
# 자신과의 유사도는 1
genres_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(genres_similarity)
# 유사도 값이 높은 영화의 제목
# 유사도 값이 높은 순으로 인덱스 값을 뽑아낸다
similar_index = np.argsort(-genres_similarity)
print(similar_index)

[[1.         0.00486926 0.         ... 0.00773342 0.         0.        ]
 [0.00486926 1.         0.         ... 0.00908723 0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.00773342 0.00908723 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[   0 2403  278 ... 1996 1961 4802]
 [   1   12  199 ... 1984 1971 4802]
 [   2   11 3285 ... 1915 1880 4802]
 ...
 [4800 2674 2108 ... 1688 1697 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [4802 2000 4066 ... 1619 1635 2401]]


# Count (빈도수)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(ngram_range=(1, 2))
count_matrix = count_vec.fit_transform(movies['keywords'])
print(count_matrix)

  (0, 9037)	1
  (0, 7010)	1
  (0, 15032)	1
  (0, 34485)	4
  (0, 40021)	2
  (0, 7433)	1
  (0, 34214)	1
  (0, 38236)	1
  (0, 15087)	1
  (0, 31412)	1
  (0, 1077)	2
  (0, 38408)	1
  (0, 27732)	1
  (0, 6161)	1
  (0, 22347)	1
  (0, 34275)	1
  (0, 3232)	1
  (0, 21496)	1
  (0, 548)	1
  (0, 1760)	1
  (0, 28328)	1
  (0, 30072)	1
  (0, 23178)	1
  (0, 1426)	1
  (0, 34431)	1
  :	:
  (4800, 41064)	1
  (4800, 19078)	1
  (4800, 9324)	1
  (4800, 2428)	1
  (4800, 24745)	1
  (4800, 33595)	1
  (4800, 21508)	1
  (4800, 2430)	1
  (4800, 14012)	1
  (4800, 9332)	1
  (4800, 28293)	1
  (4800, 33602)	1
  (4800, 24753)	1
  (4800, 19119)	1
  (4800, 36763)	1
  (4800, 28294)	1
  (4802, 8952)	1
  (4802, 25700)	1
  (4802, 11046)	1
  (4802, 15654)	1
  (4802, 5338)	1
  (4802, 11062)	1
  (4802, 25707)	1
  (4802, 5341)	1
  (4802, 8956)	1


In [6]:
# 유사도 행렬 (4803, 4803)
# 1, 1 (1번째 영화와 1번째 영화의 유사도)
# 1, 1 / 1, 2 / .... / 1, 4803
# 2, 1 / 2, 2 / .... / 2, 4803
# .....
# 4803, 1 / 4803, 2 / .... / 4803, 4803
from sklearn.metrics.pairwise import cosine_similarity

# 4803개의 영화랑 4803개의 영화끼리 유사도를 구하겠다!
genres_similarity = cosine_similarity(count_matrix, count_matrix)
print(genres_similarity)
# 유사도가 높은 영화를 알고 싶다!
# 유사도 값이 높은 것의 인덱스를 내림차순으로 출력/리턴
similar_index = np.argsort(-genres_similarity)
print(similar_index)

[[1.         0.01628008 0.         ... 0.02614435 0.         0.        ]
 [0.01628008 1.         0.         ... 0.03277368 0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.02614435 0.03277368 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[   0 2403  278 ... 1996 1961 4802]
 [   1   12 2875 ... 1984 1971 4802]
 [   2   11  147 ... 1915 1880 4802]
 ...
 [4800 2674 2108 ... 1688 1697 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [4802 2000 4774 ... 1619 1635 2401]]


In [8]:
# 사용자가 입력한 영화의 인덱스 값을 찾아내고
# similar_index 에 기록된 유사한 영화 인덱스를 찾아내고
# 유사한 영화 인덱스를 토대로 영화 이름을 찾아내면 된다!
input_movie = input()

movie_index = movies[movies['title']==input_movie].index.values
print(movie_index)
similar_movies = similar_index[movie_index, :10]
print(similar_movies)
# 인덱스로 사용하기 위해서는 1차원으로 변형해줘야하기 때문
similar_movies_index = similar_movies.reshape(-1)
print(similar_movies_index)
print(movies.iloc[similar_movies_index])

Avatar
[0]
[[   0 2403  278 4332  838  373 3158 1951 1354 3730]]
[   0 2403  278 4332  838  373 3158 1951 1354 3730]
                   title                                     genres  \
0                 Avatar   Action Adventure Fantasy Science Fiction   
2403              Aliens     Horror Action Thriller Science Fiction   
278   Planet of the Apes  Thriller Science Fiction Action Adventure   
4332      Silent Running            Adventure Drama Science Fiction   
838               Alien³              Science Fiction Action Horror   
373      Mission to Mars                            Science Fiction   
3158               Alien     Horror Action Thriller Science Fiction   
1951          Space Dogs                           Family Animation   
1354        Space Chimps                           Animation Family   
3730               Cargo           Thriller Mystery Science Fiction   

                                               keywords  
0     culture clash future space war space 